This notebook was just used for notes and other quick data and number counts.  I also used this to check the top words in each subreddit and compare them to each other.  

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import numpy as np


In [2]:
# Some of the data was importing strange, had to fix it
dtyping ={
    'title' : str,
    'url' : str,
    'author' : str 
}
df = pd.read_csv('./data/wsb_and_btc_all.csv', dtype=dtyping)
df.head()

,title,id,url,body,date_created,number_comments,author,self_post,sub,submission
0,All In The War Machine,eld4i6,https://i.redd.it/cj2vw26nmd941.jpg,NaN,1.578440e+09,780.0,everythingorange9,0.0,wsb,1
1,How to get oil back up,g55or2,https://i.redd.it/w5iqqihjo2u41.jpg,NaN,1.587461e+09,516.0,futuretrollshark,0.0,wsb,1
2,Type yy into google.,c75d5x,https://www.reddit.com/r/wallstreetbets/commen...,That’s all I’m saying.,1.561875e+09,17281.0,Alopez2897,1.0,wsb,1
3,Oil is now expenzive,d51f4o,https://i.redd.it/2j386s5iuym31.png,NaN,1.568673e+09,1019.0,NaN,0.0,wsb,1
4,"My dad, working through a 15-hour time zone di...",d29nov,https://i.redd.it/60asaz4zhsl31.jpg,NaN,1.568160e+09,1633.0,SerraTL,0.0,wsb,1


In [3]:
df.shape

(1242069, 10)

In [4]:

df['sub'].value_counts()

wsb    661080
btc    580989
Name: sub, dtype: int64

In [5]:
df[df['self_post'] == 1].shape

(286, 10)

In [6]:
df[df['self_post'] == 1]['sub'].value_counts()

wsb    168
btc    118
Name: sub, dtype: int64

In [7]:
df[df['self_post'] != 1]['sub'].value_counts()

wsb    660912
btc    580871
Name: sub, dtype: int64

In [8]:
df[df['submission'] == 1]['sub'].value_counts()

btc    990
wsb    980
Name: sub, dtype: int64

In [9]:
# df[df['sub'] == 'btc']['body'].dropna().to_clipboard(index=False)

In [10]:
# df[df['sub'] == 'wsb']['body'].dropna()[:250000].to_clipboard(index=False)

## From here and down was looking for word frequency in each subreddit.  

In [ ]:
wsb = df[df['sub'] == 'wsb'].copy()
btc = df[df['sub'] == 'btc'].copy()

In [13]:
wsb.dropna(inplace=True)
btc.dropna(inplace=True)

In [20]:
btc.head()

,title,id,url,body,date_created,number_comments,author,self_post,sub,submission
661105,I'm donating 5057 BTC to charitable causes! In...,7jj0oa,https://www.reddit.com/r/Bitcoin/comments/7jj0...,Hello! \n\nI remember staring at bitcoin a few...,1.513199e+09,2862.0,PineappleFund,1.0,btc,1
661109,Localbitcoins.com is illegally holding my 9.3 ...,98uecl,https://www.reddit.com/r/Bitcoin/comments/98ue...,&#x200B;\n\n**Edit.**\n\nThanks for all your s...,1.534807e+09,767.0,crlxzzz,1.0,btc,1
661126,Us Senate Bill S.1241 to criminalize concealed...,7iey6q,https://www.reddit.com/r/Bitcoin/comments/7iey...,"On November 28, 2017, the US Senate, Committee...",1.512768e+09,1589.0,EaFaer,1.0,btc,1
661128,Don't invest recklessly,7gi55s,https://www.reddit.com/r/Bitcoin/comments/7gi5...,I posted about this [just a few months ago](ht...,1.512030e+09,1927.0,theymos,1.0,btc,1
661131,"Guys, I am out",7fvoxh,https://www.reddit.com/r/Bitcoin/comments/7fvo...,**So I sold the last of my 22ish BTC yesterday...,1.511823e+09,1559.0,DebtFreeMFers,1.0,btc,1


In [75]:
stop_list = text.ENGLISH_STOP_WORDS.union(['https', 'com', 'www'])
# https://stackoverflow.com/questions/24386489/adding-words-to-scikit-learns-countvectorizers-stop-list/24386751#24386751


wsb_data = wsb['body'].append(wsb['title'])
btc_data = btc['body'].append(btc['title'])
cv_wsb = CountVectorizer(stop_words=stop_list)
cv_btc = CountVectorizer(stop_words=stop_list)
wsb_v = cv_wsb.fit_transform(wsb_data)
btc_v = cv_btc.fit_transform(btc_data)

# From: https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
wsb_sum = wsb_v.sum(axis = 0)
wsb_freq = [(word,wsb_sum[0,idx]) for word, idx in cv_wsb.vocabulary_.items()]
wsb_freq = sorted(wsb_freq, key = lambda x: x[1], reverse=True)


btc_sum = btc_v.sum(axis = 0)
btc_freq = [(word,btc_sum[0,idx]) for word, idx in cv_btc.vocabulary_.items()]
btc_freq = sorted(btc_freq, key = lambda x: x[1], reverse=True)


In [78]:
words = []
for word, freq in wsb_freq:
    words.append({
        'word' : word,
        'wsbfreq' : freq
    })
for word, freq in wsb_freq:
    words.append({
    'word' : word,
    'btcfreq' : freq
})
wordfreq = pd.DataFrame(words)
wordfreq.head()

,word,wsbfreq,btcfreq
0,money,138.0,NaN
1,just,134.0,NaN
2,market,133.0,NaN
3,time,125.0,NaN
4,don,109.0,NaN


In [79]:
wordfreq.fillna(0, inplace=True)
wordfreq.shape

(12830, 3)

In [80]:
wordfreq['wsbpercentile'] = wordfreq['wsbfreq'].rank(pct=True)
wordfreq['btcpercentile'] = wordfreq['btcfreq'].rank(pct=True)

In [81]:
per = .25
wordfreq[(wordfreq['wsbpercentile'] > per) & (wordfreq['btcpercentile'] > per)]
# http://www.datasciencemadesimple.com/percentile-rank-column-pandas-python-2/

,word,wsbfreq,btcfreq,wsbpercentile,btcpercentile
0,money,138.0,0.0,1.000000,0.250039
1,just,134.0,0.0,0.999922,0.250039
2,market,133.0,0.0,0.999844,0.250039
3,time,125.0,0.0,0.999766,0.250039
4,don,109.0,0.0,0.999649,0.250039
...,...,...,...,...,...
12825,investigators,0.0,1.0,0.250039,0.636789
12826,incomplete,0.0,1.0,0.250039,0.636789
12827,visited,0.0,1.0,0.250039,0.636789
12828,packed,0.0,1.0,0.250039,0.636789


In [74]:
wordfreq.sort_values('wsbfreq')

,word,wsbfreq,btcfreq,wsbpercentile,btcpercentile
13355,items,0.0,1.0,0.250037,0.632038
8896,discussions,0.0,2.0,0.250037,0.801962
8897,complaint,0.0,2.0,0.250037,0.801962
8898,contain,0.0,2.0,0.250037,0.801962
8899,incurred,0.0,2.0,0.250037,0.801962
...,...,...,...,...,...
4,in,699.0,0.0,0.999701,0.250037
3,of,931.0,0.0,0.999775,0.250037
2,and,958.0,0.0,0.999850,0.250037
1,to,1390.0,0.0,0.999925,0.250037


In [83]:
cv_wsb.vocabulary_['money']

3872